In [1]:
from qch2025.pkg.models.RNN import network as network
from qch2025.pkg.models.RNN import func as func

from qch2025.pkg.dataset import DS

import qch2025.pkg.plotting as plt

import numpy as np
import pandas as pd
import torch
import datetime

n_epochs = 50
window_size = 256
batch_size = 32
window_steps = 16
attn_heads = 4

learning_rate = 0.0001
dropout = 0.2

dtype = torch.float32

# Training
train_dataset = DS(dataset_path="/home/ubuntu/repos/quantchallenge-2025/qch2025/dataset/train.csv",
                        window_size=window_size,
                        window_steps=window_steps,
                        dtype=dtype)

rnn = network.RNN(14, 512, 2, dropout, attn_heads, 2, device=torch.device("cuda"), dtype=dtype)
losses = func.train(rnn, dataset=train_dataset, epochs=n_epochs, learning_rate=learning_rate)

plt.plot_line(np.arange(1, n_epochs+1), losses)


#checkpoint = torch.load("/home/ubuntu/repos/quantchallenge-2025/weights/weights.pth", weights_only=True)
#rnn.load_state_dict(checkpoint['model_state_dict'])

eval = DS(dataset_path="/home/ubuntu/repos/quantchallenge-2025/qch2025/dataset/test.csv",
                    window_size=window_size,
                    window_steps=1,
                    eval=True,
                    device=torch.device("cuda"),
                    dtype=dtype)
print(eval.ids.shape, eval.train.shape)
y1, y2, ids = func.eval(rnn, eval)

df = pd.DataFrame({"id": ids, "Y1": y1, "Y2": y2})
df["id"] = df["id"] + 1
df.to_csv(f"/home/ubuntu/repos/quantchallenge-2025/qch2025/outputs/{datetime.datetime.now()}_predicted.csv")






torch.Size([80000, 14])
torch.Size([80000, 2])
Initialized dataset torch.Size([4484, 256, 14]) torch.Size([4484, 256, 2])
torch.Size([8192, 2]) torch.Size([8192, 2])
torch.Size([])
torch.Size([2]) torch.Size([2]) torch.Size([])
torch.Size([1]) tensor([-0.0052], device='cuda:0', grad_fn=<SubBackward0>)
[2025-09-21 11:48:35.298269] Current batch item: 0, took 312 ms, loss: -0.005196869373321533, mean loss: -0.005196869373321533
torch.Size([8192, 2]) torch.Size([8192, 2])
torch.Size([])
torch.Size([2]) torch.Size([2]) torch.Size([])
torch.Size([1]) tensor([-0.0246], device='cuda:0', grad_fn=<SubBackward0>)


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.